<a href="https://colab.research.google.com/github/hfelizzola/Investigaciones-de-Operaciones-I/blob/main/flujo-redes/03_modelo_transporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de Transporte

In [ ]:
!pip install gamspy --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.1/215.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 73.7 MB/s eta 0:00:00


## Problema de Distribución

Uno de los productos más importantes de la P & T COMPANY es el chícharo enlatado. Los chicharos se preparan en tres enlatadoras:  Bellingham, Eugene y Albert Lea.  Después se envían por camión a cuatro almacenes de distribución: Sacramento, Salt Lake City, Rapid City y Albuquerque. Debido a que los costos de transporte constituyen un gasto importante, la administración ha iniciado un estudio para reducirlos a su mínima expresión. Se ha estimado la capacidad de producción de cada enlatadora durante la próxima temporada y se ha asignado a cada almacén cierta cantidad de la producción total de chicharos, esto dependiente la demanda de la zona de influencia.

<table class="w-full">
    <thead>
        <tr>
            <th class="rounded-tl-xl"></th>
            <th>Almacén 1</th>
            <th>Almacén 2</th>
            <th>Almacén 3</th>
            <th>Almacén 4</th>
            <th class="rounded-tr-xl">Capacidad de Producción</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td class="font-bold">Enlatadora 1</td>
            <td>464</td>
            <td>513</td>
            <td>654</td>
            <td>867</td>
            <td>75</td>
        </tr>
        <tr>
            <td class="font-bold">Enlatadora 2</td>
            <td>352</td>
            <td>416</td>
            <td>690</td>
            <td>791</td>
            <td>125</td>
        </tr>
        <tr>
            <td class="font-bold">Enlatadora 3</td>
            <td class="rounded-bl-xl">995</td>
            <td>682</td>
            <td>588</td>
            <td>685</td>
            <td>100</td>
        </tr>
        <tr>
            <td class="font-bold">Asignación</td>
            <td>80</td>
            <td>65</td>
            <td>70</td>
            <td>85</td>
            <td></td>
        </tr>
    </tbody>
</table>

**Modelo Matemático**

- Conjuntos:
  - $i$: Nodos de origen, $I = \{1,2,3\}$
  - $j$: Nodos de destino, $J = \{1,2,3\}$
- Parámetros:
  - $c_{ij}$: Costo de transporte de $i \to j$
  - $s_i$: Capacidad del nodo $i$
  - $d_j$: Demanda del nodo $j$
- Variable de decisión:
  - $x_{ij}$: Cantidad a enviar de $i \to j$
- Función Objetivo y Restricciones:

\begin{equation*}
\text{Min. } Z = \sum_{i=1}^{m}\sum_{j=1}^{n} c_{ij}x_{ij}
\end{equation*}
\begin{align*}
SA:\\
\sum_{j \in J} x_{ij} &= s_i && \forall i \in I\\
\sum_{i \in I} x_{ij} &= d_j && \forall j \in J\\
x_{ij} &\ge 0, && \forall i \in I, \forall j \in J\\\\
\end{align*}

In [ ]:
import numpy as np
import pandas as pd
from gamspy import Container, Set, Variable, Parameter, Equation, Model, Sense, Sum, Options

In [ ]:
m = Container()

# Conjuntos
planta = Set(m, name='planta', records=['P1','P2','P3'])
almacen = Set(m, name='almacen', records=['A1','A2','A3','A4'])

# Variable de decisión
x = Variable(m, name='x', type='Positive', domain=[planta,almacen])

# Parametros
capacidad = Parameter(m, name='capacidad', domain=[planta], records=np.array([75,125,100]))
demanda = Parameter(m, name='demanda', domain=[almacen], records=np.array([80,65,70,85]))
costo = Parameter(m, name='costo', domain=[planta,almacen], records=np.array([[464,513,654,867],
                                                                              [352,416,690,791],
                                                                              [995,682,388,685]]))

# Restricciones
capacidad_max = Equation(m, name='capacidad_max', domain=[planta])
capacidad_max[planta] = Sum(almacen, x[planta,almacen]) == capacidad[planta]

demanda_max = Equation(m, name='demanda_max', domain=[almacen])
demanda_max[almacen] = Sum(planta, x[planta,almacen]) == demanda[almacen]

costo_total = Sum([planta,almacen], costo[planta,almacen] * x[planta,almacen])

# Model
modelo_transporte = Model(m, name='modelo_transporte', objective=costo_total, sense=Sense.MIN, problem='LP', equations=[capacidad_max, demanda_max])
modelo_transporte.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,152535.0,8,13,LP,CPLEX,0.0


In [ ]:
x.records.pivot(index="planta", columns="almacen", values='level')

almacen,A1,A2,A3,A4
planta,,,,
P1,0.0,20.0,0.0,55.0
P2,80.0,45.0,0.0,0.0
P3,0.0,0.0,70.0,30.0


In [ ]:
capacidad_max.records

,planta,level,marginal,lower,upper,scale
0,P1,75.0,97.0,75.0,75.0,1.0
1,P2,125.0,0.0,125.0,125.0,1.0
2,P3,100.0,-85.0,100.0,100.0,1.0


In [ ]:
demanda_max.records

,almacen,level,marginal,lower,upper,scale
0,A1,80.0,352.0,80.0,80.0,1.0
1,A2,65.0,416.0,65.0,65.0,1.0
2,A3,70.0,473.0,70.0,70.0,1.0
3,A4,85.0,770.0,85.0,85.0,1.0


## Modelo de Transporte no Balaceado

MG Auto cuenta con tres plantas en Los Ángeles, Detroit y Nueva Orleans, y dos importantes centros de distribución en Denver y Miami. Las capacidades trimestrales de las tres plantas son 1000, 1300 y 1200 automóviles, y las demandas de los dos centros de distribución durante el mismo periodo son de 2300 y 1400 automóviles. La distancia en millas entre las plantas y los centros de distribución aparece en la tabla.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg"><thead>
  <tr>
    <th class="tg-0lax" colspan="2" rowspan="3">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black"> </span>&nbsp;&nbsp;&nbsp;&nbsp;<br> <br> </th>
    <th class="tg-0lax" colspan="2">&nbsp;&nbsp;&nbsp;<br>&nbsp;&nbsp;&nbsp;&nbsp;</th>
    <th class="tg-0lax" rowspan="3">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black">Capacidad</span>&nbsp;&nbsp;&nbsp;</th>
  </tr>
  <tr>
    <th class="tg-0lax" colspan="2">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black">Centros&nbsp;&nbsp;&nbsp;de Distribución</span>&nbsp;&nbsp;&nbsp;</th>
  </tr>
  <tr>
    <th class="tg-0lax">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black">Denver</span>&nbsp;&nbsp;&nbsp;</th>
    <th class="tg-0lax">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black">Miami</span>&nbsp;&nbsp;&nbsp;</th>
  </tr></thead>
<tbody>
  <tr>
    <td class="tg-0lax" rowspan="3">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black">Plantas</span>&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black">Los&nbsp;&nbsp;&nbsp;Angeles</span>&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br> <span style="font-weight:normal;font-style:normal;color:black">$80 </span>&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br> <span style="font-weight:normal;font-style:normal;color:black">$215 </span>&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black">1000</span>&nbsp;&nbsp;&nbsp;</td>
  </tr>
  <tr>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black">Detroit</span>&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br> <span style="font-weight:normal;font-style:normal;color:black">$100 </span>&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br> <span style="font-weight:normal;font-style:normal;color:black">$108 </span>&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black">1300</span>&nbsp;&nbsp;&nbsp;</td>
  </tr>
  <tr>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black">Nueva&nbsp;&nbsp;&nbsp;Orleans</span>&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br> <span style="font-weight:normal;font-style:normal;color:black">$102 </span>&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br> <span style="font-weight:normal;font-style:normal;color:black">$68 </span>&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black">1200</span>&nbsp;&nbsp;&nbsp;</td>
  </tr>
  <tr>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black"> </span>&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black">Demanda</span>&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black">2300</span>&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black">1400</span>&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0lax">&nbsp;&nbsp;&nbsp;<br><span style="font-weight:normal;font-style:normal;color:black"> </span>&nbsp;&nbsp;&nbsp;</td>
  </tr>
</tbody></table>

Observe que en este caso la demanda (3700) supera la capacidad (3500), esto es:

$$
\sum_{i \in I} s_i < \sum_{j \in J} d_j
$$

Por tanto, el modelo matemático se debe modificar colocando una desigualdad de $\le$ en la restricción de demanda:

\begin{equation*}
\text{Min. } Z = \sum_{i=1}^{m}\sum_{j=1}^{n} c_{ij}x_{ij}
\end{equation*}
\begin{align*}
SA:\\
\sum_{j \in J} x_{ij} &= s_i && \forall i \in I\\
\sum_{i \in I} x_{ij} &\le d_j && \forall j \in J\\
x_{ij} &\ge 0, && \forall i \in I, \forall j \in J\\\\
\end{align*}



In [ ]:
m = Container()

# Conjuntos
planta = Set(m, name='planta', records=['Los Angeles','Detroit','Nueva Orleans'])
centros = Set(m, name='centros', records=['Denver','Miami'])

# Variable de decisión
x = Variable(m, name='x', type='Positive', domain=[planta,centros])

# Parametros
capacidad = Parameter(m, name='capacidad', domain=[planta], records=np.array([1000,1300,1200]))
demanda = Parameter(m, name='demanda', domain=[centros], records=np.array([2300,1400]))
costo = Parameter(m, name='costo', domain=[planta,centros], records=np.array([[80,215],
                                                                              [100,108],
                                                                              [102,68]]))

# Restricciones
capacidad_max = Equation(m, name='capacidad_max', domain=[planta])
capacidad_max[planta] = Sum(centros, x[planta,centros]) == capacidad[planta]

demanda_max = Equation(m, name='demanda_max', domain=[centros])
demanda_max[centros] = Sum(planta, x[planta,centros]) <= demanda[centros]

costo_total = Sum([planta,centros], costo[planta,centros] * x[planta,centros])

# Model
modelo_transporte = Model(m, name='modelo_transporte', objective=costo_total, sense=Sense.MIN, problem='LP', equations=[capacidad_max, demanda_max])
modelo_transporte.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,291600.0,6,7,LP,CPLEX,0.0


In [ ]:
x.records.pivot(index="planta", columns="centros", values='level')

centros,Denver,Miami
planta,,
Los Angeles,1000.0,0.0
Detroit,1300.0,0.0
Nueva Orleans,0.0,1200.0


In [ ]:
capacidad_max.records

,planta,level,marginal,lower,upper,scale
0,Los Angeles,1000.0,80.0,1000.0,1000.0,1.0
1,Detroit,1300.0,100.0,1300.0,1300.0,1.0
2,Nueva Orleans,1200.0,68.0,1200.0,1200.0,1.0


In [ ]:
demanda_max.records

,centros,level,marginal,lower,upper,scale
0,Denver,2300.0,0.0,-inf,2300.0,1.0
1,Miami,1200.0,0.0,-inf,1400.0,1.0


## ¿Donde Aumentar la capacidad para suplir la demanda en Miami?

In [ ]:
costo_total_base = 291600

In [ ]:
# Escenario 1: Aumentar en 200 unidades la capacidad de los Angeles
costo_escenario_1 = 309200
(costo_escenario_1 - costo_total_base)/200

88.0

In [ ]:
# Escenario 2: Aumentar en 200 unidades la capacidad de Detroit
costo_escenario_2 = 313200
(costo_escenario_2 - costo_total_base)/200

108.0

In [ ]:
# Escenario 3: Aumentar en 200 unidades la capacidad de Nueva Orleans
costo_escenario_3 = 305200
(costo_escenario_3 - costo_total_base)/200

68.0

## Modelo No Balanceado: creando nodo ficticio

In [ ]:
m = Container()

# Conjuntos
planta = Set(m, name='planta', records=['Los Angeles','Detroit','Nueva Orleans','Ficticia'])
centros = Set(m, name='centros', records=['Denver','Miami'])

# Variable de decisión
x = Variable(m, name='x', type='Positive', domain=[planta,centros])

# Parametros
capacidad = Parameter(m, name='capacidad', domain=[planta], records=np.array([1000,1300,1200,200]))
demanda = Parameter(m, name='demanda', domain=[centros], records=np.array([2300,1400]))
costo = Parameter(m, name='costo', domain=[planta,centros], records=np.array([[80,215],
                                                                              [100,108],
                                                                              [102,68],
                                                                              [0,0]]))

# Restricciones
capacidad_max = Equation(m, name='capacidad_max', domain=[planta])
capacidad_max[planta] = Sum(centros, x[planta,centros]) == capacidad[planta]

demanda_max = Equation(m, name='demanda_max', domain=[centros])
demanda_max[centros] = Sum(planta, x[planta,centros]) == demanda[centros]

costo_total = Sum([planta,centros], costo[planta,centros] * x[planta,centros])

# Model
modelo_transporte = Model(m, name='modelo_transporte', objective=costo_total, sense=Sense.MIN, problem='LP', equations=[capacidad_max, demanda_max])
modelo_transporte.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,291600.0,7,9,LP,CPLEX,0.0


In [ ]:
x.records.pivot(index="planta", columns="centros", values='level')

centros,Denver,Miami
planta,,
Los Angeles,1000.0,0.0
Detroit,1300.0,0.0
Nueva Orleans,0.0,1200.0
Ficticia,0.0,200.0


## Modelo de Producción

In [ ]:
m = Container()

# Conjuntos
mes_produccion = Set(m, name='mes_produccion', records=['Marzo','Abril','Mayo','Junio'])
mes_demanda = Set(m, name='mes_demanda', records=['Marzo','Abril','Mayo','Junio'])

# Variable de decisión: cantidad a fabricar en el mes de producción y entregar el mes de demanda
x = Variable(m, name='x', type='Positive', domain=[mes_produccion,mes_demanda])

# Parametros
capacidad = Parameter(m, name='capacidad', domain=[mes_produccion], records=np.array([50,180,280,270]))
demanda = Parameter(m, name='demanda', domain=[mes_demanda], records=np.array([100,200,180,300]))
costo = Parameter(m, name='costo', domain=[mes_produccion,mes_demanda], records=np.array([[40,40.5,41,41.5],
                                                                                          [42,40,40.5,41],
                                                                                          [44,42,40,40.5],
                                                                                          [46,44,42,40]]))

# Restricciones
capacidad_max = Equation(m, name='capacidad_max', domain=[mes_produccion])
capacidad_max[mes_produccion] = Sum(mes_demanda, x[mes_produccion,mes_demanda]) == capacidad[mes_produccion]

demanda_max = Equation(m, name='demanda_max', domain=[mes_demanda])
demanda_max[mes_demanda] = Sum(mes_produccion, x[mes_produccion,mes_demanda]) == demanda[mes_demanda]

costo_total = Sum([mes_produccion,mes_demanda], costo[mes_produccion,mes_demanda] * x[mes_produccion,mes_demanda])

# Model
modelo_transporte = Model(m, name='modelo_transporte', objective=costo_total, sense=Sense.MIN, problem='LP', equations=[capacidad_max, demanda_max])
modelo_transporte.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,31455.0,9,17,LP,CPLEX,0.0


In [ ]:
x.records.pivot(index="mes_produccion", columns="mes_demanda", values='level')

mes_demanda,Marzo,Abril,Mayo,Junio
mes_produccion,,,,
Marzo,50.0,0.0,0.0,0.0
Abril,0.0,180.0,0.0,0.0
Mayo,50.0,20.0,180.0,30.0
Junio,0.0,0.0,0.0,270.0


In [ ]:
capacidad_max.records

,mes_produccion,level,marginal,lower,upper,scale
0,Marzo,50.0,36.5,50.0,50.0,1.0
1,Abril,180.0,38.5,180.0,180.0,1.0
2,Mayo,280.0,40.5,280.0,280.0,1.0
3,Junio,270.0,40.0,270.0,270.0,1.0


## Ejercicio Refineria (5.8)

In [ ]:
m = Container()

# Conjuntos
planta = Set(m, name='planta', records=['R1','R2','R3'])
almacen = Set(m, name='almacen', records=['A1','A2','A3'])

# Variable de decisión
x = Variable(m, name='x', type='Positive', domain=[planta,almacen])

# Parametros
M = 100
capacidad = Parameter(m, name='capacidad', domain=[planta], records=np.array([6,5,8]))
demanda = Parameter(m, name='demanda', domain=[almacen], records=np.array([4,8,7]))
costo = Parameter(m, name='costo', domain=[planta,almacen], records=np.array([[1.2,1.8,M],
                                                                              [3.0,1.0,0.8],
                                                                              [2.0,2.5,1.2]]))

# Restricciones
capacidad_max = Equation(m, name='capacidad_max', domain=[planta])
capacidad_max[planta] = Sum(almacen, x[planta,almacen]) == capacidad[planta]

demanda_max = Equation(m, name='demanda_max', domain=[almacen])
demanda_max[almacen] = Sum(planta, x[planta,almacen]) == demanda[almacen]

costo_total = Sum([planta,almacen], costo[planta,almacen] * x[planta,almacen])

# Model
modelo_transporte = Model(m, name='modelo_transporte', objective=costo_total, sense=Sense.MIN, problem='LP', equations=[capacidad_max, demanda_max])
modelo_transporte.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,24.3,7,10,LP,CPLEX,0.0


In [ ]:
x.records.pivot(index="planta", columns="almacen", values='level')

almacen,A1,A2,A3
planta,,,
R1,4.0,2.0,0.0
R2,0.0,5.0,0.0
R3,0.0,1.0,7.0
